In [3]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load the Diagnostics.xlsx data
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
diagnostics_df = pd.read_excel(diagnostics_file)

# Rename "SA" to "SI" in the "Rhythm" column
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].replace("SA", "SI")

# Drop rows with any missing values
diagnostics_df = diagnostics_df.dropna()

# Encode "Gender" column: 0 for "MALE" and 1 for "FEMALE"
diagnostics_df["Gender"] = diagnostics_df["Gender"].map({"MALE": 0, "FEMALE": 1})

# Merge specified labels
merge_mapping = {
    "AF": "AFIB", "AFIB": "AFIB",
    "SVT": "GSVT", "AT": "GSVT", "SAAWR": "GSVT", "ST": "GSVT", "AVNRT": "GSVT", "AVRT": "GSVT",
    "SB": "SB",
    "SR": "SR", "SI": "SR"
}
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].map(merge_mapping)

# Separate features and labels
features = diagnostics_df.drop(columns=["FileName", "Rhythm", "Beat"]).values
labels = diagnostics_df["Rhythm"].values  # Using "Rhythm" as the target variable

# Convert features to float32
features = features.astype("float32")

# Encode labels as one-hot with merged classes
unique_labels = np.unique(labels)
label_map = {label: index for index, label in enumerate(unique_labels)}
labels_encoded = np.array([label_map[label] for label in labels])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# Save X_test to a file
np.savetxt("X_test.txt", X_test, fmt='%f', delimiter=' ')

# Save y_test to a separate file
np.savetxt("y_test.txt", y_test, fmt='%d')  # Use '%d' if y_test contains integers

In [4]:
print(X_test[1])

[ 68.   0.  67. 250.  76. 482. 509.  75.  82.  11. 219. 257. 460.]


In [5]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model
dt_classifier.fit(X_train, y_train)

# Predict on test data
y_pred = dt_classifier.predict(X_test)

# Map back to original labels for a readable report
label_names = [label for label, index in sorted(label_map.items(), key=lambda item: item[1])]

# Evaluate and print classification report
print("\nClassification Report (Decision Tree):\n")
print(classification_report(y_test, y_pred, target_names=label_names, digits=5))


Classification Report (Decision Tree):

              precision    recall  f1-score   support

        AFIB    0.69007   0.67217   0.68100       424
        GSVT    0.79352   0.81328   0.80328       482
          SB    0.97823   0.98327   0.98074       777
          SR    0.88009   0.87025   0.87514       447

    accuracy                        0.85915      2130
   macro avg    0.83548   0.83474   0.83504      2130
weighted avg    0.85848   0.85915   0.85876      2130



In [6]:
print(dt_classifier.get_depth())
print(dt_classifier.get_n_leaves())

24
689


In [7]:
# Time  to Generating cpp header file

from micromlgen import port

converted_c_code = port(dt_classifier)

# Now you can save the code
with open("MicroGencode/author_provided_feat_dt_v1.h",
          "w") as modelFile:
    modelFile.write(converted_c_code)